In [1]:
import folium
import requests
import json

In [2]:
# let's start by creating an empty map using the IMO basemapbas

# basemap tile url and attribution
basemap_tiles = "https://geo.vedur.is/geoserver/www/imo_basemap_epsg3857/{z}/{x}/{y}.png"
basemap_attribution = "Icelandic Met Office | National Land Survey of Iceland | © OpenStreetMap contributors"

#create map and assign options
map = folium.Map(location=[65, -19], zoom_start=6, tiles=basemap_tiles, attr=basemap_attribution)

#initiate and display map. You can also save as html with map.save("index.html")
map

In [3]:
# let's add a WMS layers and zoom in our map a bit
# we will use on of several glacier outlines available
# to get a full list of avaialble layers goto https://geo.vedur.is/geoserver/wms?request=GetCapabilities&service=WMS

map = folium.Map(location=[66.15, -22.2], zoom_start=11, tiles=basemap_tiles, attr=basemap_attribution)

# define parameters for a WMS layer
wms_overlay = folium.raster_layers.WmsTileLayer(
               url="https://geo.vedur.is/geoserver/wms", # url to wms server
               name="Glacier outlines around 1890", # name of wms layer for layer controls
               fmt='image/png',  # image/png is the default for format
               layers="glaciology:glacieroutline_lia", # name of wms layer
               transparent=True, # allow transparancy
               #opacity=0.5, # set transparancy
               overlay=True, # overlay wms layer on top of basemap
               )

wms_overlay.add_to(map)

map

In [4]:
base_url = 'https://geo.vedur.is/geoserver/ows?'
params = {
    'service':'WFS',
    'version': '1.0.0',
    'request': 'GetFeature',
    'typeName': 'meteorology:IcelandicMetOffice_CAP_areas',
    'outputFormat': 'application/json'
}

r = requests.get(base_url, params)
r.json()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'IcelandicMetOffice_CAP_areas.1',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-24.01, 65.71],
       [-23.65, 65.59],
       [-23.39, 65.6],
       [-23.01, 65.65],
       [-22.54, 65.69],
       [-22, 65.7],
       [-21.57, 65.42],
       [-21.5, 64.92],
       [-22.81, 64.89],
       [-24.03, 64.82],
       [-24.06, 64.91],
       [-22.62, 65.14],
       [-22.46, 65.44],
       [-24.13, 65.37],
       [-24.58, 65.5],
       [-24.11, 65.81],
       [-24.01, 65.71]]]]},
   'geometry_name': 'the_geom',
   'properties': {'name_en': 'Breidafjordur',
    'name_is': 'Breiðafjörður',
    'region_ord': 4}},
  {'type': 'Feature',
   'id': 'IcelandicMetOffice_CAP_areas.2',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-16.82, 65.55],
       [-16.45, 65.53],
       [-15.66, 65.54],
       [-14.59, 64.79],
       [-14.77, 64.59],
       [-15.8, 64.4],
       [-16.71, 64.3],
       [-17.82,

In [5]:
# r.encoding = 'ISO-8859-1'
data = r.json() # add data to get nested list
with open('forecast_areas.json', 'w') as f:
    json.dump(data, f)
f.close()

In [6]:
map = folium.Map(location=[65, -19], zoom_start=6, tiles=basemap_tiles, attr=basemap_attribution)
folium.GeoJson(data, name="Forecast areas").add_to(map)
folium.LayerControl().add_to(map)
map